In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=5174774de66c8ce9bb470e655de025b8deebcfb4b93ad542d7d6568784dfc1f7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer
model_1 = SentenceTransformer('all-MiniLM-L6-v2')

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
#import modules
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM,Embedding,Dropout, add


In [4]:
BASE_DIR= '/kaggle/input/flickr8k'
WORKING_DIR='/kaggle/working/'

## FEATURE EXTRACTION 

In [5]:
#LOAD VGG16
model=VGG16()
model=Model(inputs=model.inputs, outputs=model.layers[-2].output)
print(model.summary())

553467096/553467096 [==============================] - 3s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [6]:
##extraction

features={}
directory='/kaggle/input/flickr8k/Images'

for img_name in tqdm(os.listdir(directory)):
    img_path= directory+'/'+ img_name
    img=load_img(img_path,target_size=(224,224))
    img=img_to_array(img)
    img=img.reshape(1,img.shape[0],img.shape[1],img.shape[2])
    img=preprocess_input(img)
    feature=model.predict(img,verbose=0)
    image_id=img_name.split('.')[0]
    #store features
    features[image_id]=feature
    
directory='/kaggle/input/stable-diffusion-image-to-prompts/images'

for img_name in tqdm(os.listdir(directory)):
    img_path= directory+'/'+ img_name
    img=load_img(img_path,target_size=(224,224))
    img=img_to_array(img)
    img=img.reshape(1,img.shape[0],img.shape[1],img.shape[2])
    img=preprocess_input(img)
    feature=model.predict(img,verbose=0)
    image_id=img_name.split('.')[0]
    #store features
    features[image_id]=feature


  0%|          | 0/8091 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [7]:
#storing features in pickle
pickle.dump(features,open(os.path.join(WORKING_DIR,'features.pkl'),'wb'))

In [8]:
#load from pickle
with open(os.path.join(WORKING_DIR,'features.pkl') , "rb") as f:
    features=pickle.load(f)

## LOADING CAPTION DATA


In [9]:
with open('/kaggle/input/flickr8k/captions.txt', 'r') as f:
    next(f)
    captions_doc=f.read()
    

In [10]:
import csv
#mapping image to caption
mapping={}
for line in tqdm(captions_doc.split('\n')):
    tokens=line.split(',')
    if len(line)<2:
        continue
    image_id,caption= tokens[0],tokens[1:]
    image_id=image_id.split('.')[0]
    caption=" ".join(caption)
    
    if image_id not in mapping:
        mapping[image_id]=[]
    
    mapping[image_id].append(caption)
    
    

filename = "/kaggle/input/stable-diffusion-image-to-prompts/prompts.csv"  # Replace with the actual filename

with open(filename, 'r') as file:
    reader = csv.reader(file)
    for i, row in enumerate(reader):
        if i != 0 and row:  # Skip the first row and empty rows
            if row[0] not in mapping:
                mapping[row[0]]=[]
                
            mapping[row[0]].append(row[1])
            
        
             





  0%|          | 0/40456 [00:00<?, ?it/s]

In [11]:
def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            caption=captions[i]
            #preprocessing 
            caption=caption.lower()
            caption=caption.replace('[A-Za-z]','')
            caption=caption.replace('\s+',' ')
            caption='startseq '+" ".join([word for word in caption.split() if len(word)>1])+ ' endseq'
            captions[i]=caption

In [12]:
# preprocess 
clean(mapping)

In [13]:
all_captions=[]
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [14]:
#tokenization
tokenizer=Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size=len(tokenizer.word_index)+1

In [15]:
max_length=max(len(caption.split()) for caption in all_captions)
max_length

35

# #train test split


In [16]:
image_ids=list(mapping.keys())
split=int(len(image_ids)*0.9)
train=image_ids[:split]
test=image_ids[split:]

In [21]:
#data generator
def data_generator(data_keys,mapping,features,tokenizer, max_length, vocab_size, batch_size):
    X1,X2,y=list(),list(),list()
    n=0
    while 1:
        for key in data_keys:
            n+=1
            captions=mapping[key]
            for caption in captions:
                #encode the seq
                seq=tokenizer.texts_to_sequences([caption])[0]
                #split into X,y pairs
                for i in range(1,len(seq)):
                    in_seq,out_seq=seq[:i],seq[i]
                    #pad input seq
                    in_seq=pad_sequences([in_seq],maxlen=max_length)[0]
                    #encode seq
                    out_seq=to_categorical([out_seq],num_classes=vocab_size)[0]
                    
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n==batch_size:
                X1,X2,y=np.array(X1),np.array(X2),np.array(y)
                yield [X1,X2] , y
                X1,X2,y=list(),list(),list()
                n=0

## model creation

In [19]:
#encoder model
#image feature layers
inputs1=Input(shape=(4096,))
f1=Dropout(0.4)(inputs1)
f2=Dense(256,activation='relu')(f1)

#seq feature layer
inputs2=Input(shape=(max_length,))
s1=Embedding(vocab_size,256,mask_zero=True)(inputs2)
s2=Dropout(0.4)(s1)
s3=LSTM(256)(s2)

#decoder model
decoder1=add([f2,s3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)

model=Model(inputs=[inputs1,inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

#plot_model(model,show_shapes=True)

In [20]:
#train the model
epochs=18
batch_size=64
steps=len(train)//batch_size

for i in range(epochs):
    generator=data_generator(train, mapping,features,tokenizer, max_length, vocab_size, batch_size)
    model.fit(generator,epochs=1,steps_per_epoch=steps, verbose=1)


113/113 [==============================] - 63s 552ms/step - loss: 2.3791


In [21]:
model.save(WORKING_DIR+'/my_model_1.h5')

## generate caption

In [17]:
#load model
from tensorflow.keras.models import load_model

In [18]:
model=load_model('/kaggle/working/my_model_1.h5')

In [19]:
def idx_to_word(integer,tokenizer):
    for word,index in tokenizer.word_index.items():
        if index==integer:
            return word
    return None
            

In [20]:
def caption_prediction(model,image,tokenizer,max_length):
    in_text='startseq'
    for i in range(max_length):
        #encode input seq
        sequence=tokenizer.texts_to_sequences([in_text])[0]
        #pad the seq
        sequence=pad_sequences([sequence],max_length)
        #predict next word
        yhat=model.predict([image,sequence],verbose=0)
        yhat= np.argmax(yhat)
        word=idx_to_word(yhat,tokenizer)
        
        if word is None:
            break
        in_text+=" "+ word
        if word=='endseq':
            break
    return in_text

In [21]:
from nltk.translate.bleu_score import corpus_bleu
#validate with text data
actual, predicted= list(),list()

for key in tqdm(test):
    #get actual caption
    captions= mapping[key]
    #predicted caption
    ypred=caption_prediction(model,features[key],tokenizer,max_length)
    actual_captions=[caption.split() for caption in captions]
    ypred=ypred.split()
    actual.append(actual_captions)
    predicted.append(ypred)
    
 #bleu score
print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0,0,0,0)))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5,0.5,0,0)))

  0%|          | 0/810 [00:00<?, ?it/s]

BLEU-1: 0.539385
BLEU-2: 0.317103


## visualize the result

In [21]:
from PIL import Image
import matplotlib.pyplot as plt
def generate_caption(image_name):
    image_path=os.path.join("/kaggle/input/stable-diffusion-image-to-prompts/images",image_name)
    image_id=image_name.split('.')[0]
    #image= Image.open(image_path)
    #captions=mapping[image_id]
    ypred=caption_prediction(model,features[image_id],tokenizer,max_length)
    return ypred 

In [ ]:
#load model
from tensorflow.keras.models import load_model
model_1=load_model('/kaggle/input/my-model/my_model_1.h5')

In [22]:

sentences=[]
directory='/kaggle/input/stable-diffusion-image-to-prompts/images'

for img_name in tqdm(os.listdir(directory)):
    sentences.append(generate_caption(img_name))


embeddings = model_1.encode(sentences)
print(embeddings)


  0%|          | 0/7 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[-0.05959426  0.03174604 -0.06907127 ... -0.08785734 -0.04295547
   0.03891272]
 [-0.06589831  0.06656386  0.04427902 ... -0.01740234  0.00573433
   0.04439513]
 [ 0.01826437  0.0884907  -0.02657704 ...  0.03458299  0.00778888
   0.06927446]
 ...
 [-0.00463759  0.08915918  0.02655171 ...  0.02230095  0.07641359
  -0.03694128]
 [-0.02347468  0.01386189  0.08982924 ...  0.01323435  0.05274043
   0.05073986]
 [-0.04374108  0.01773241  0.04756919 ... -0.02921897  0.00483093
  -0.03782266]]


In [27]:
directory='/kaggle/input/stable-diffusion-image-to-prompts/images'
list_img=[]

for img_name in tqdm(os.listdir(directory)):
    image_id=img_name.split('.')[0]
    list_img.append(image_id)

  0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
list_img.sort()



['20057f34d',
 '227ef0887',
 '92e911621',
 'a4e1c55a9',
 'c98f79f71',
 'd8edf2e40',
 'f27825b2c']

In [33]:

def save_embeddings_to_csv(embeddings_list, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['img_id', '#values'])
        for image_id, embeddings in enumerate(embeddings_list, start=0):
            for index, embedding in enumerate(embeddings):
                writer.writerow([f'{list_img[image_id]}_{index+1}', embedding])

save_embeddings_to_csv(embeddings, 'submission.csv')
